In [1]:
import pandas as pd
import re

In [2]:
# Read in the raw job data from a CSV file and store it in a DataFrame
jobs_df_raw = pd.read_csv('jobs.csv')

In [3]:
# Define a function to extract the company name from a job link
def isolate_company_name(ex):
    pattern = r"(?<=www\.notion\.so\/)[\w\s-]+(?=-)"  # regex pattern to match the company name
    result = re.search(pattern, ex).group()  # search for the pattern in the job link and extract the matched substring
    ret = result.replace("-"," ")
    return ret  # return the extracted company name

# Apply the isolate_company_name function to the 'Company' column of the DataFrame to create a new 'Company Name' column
jobs_df_raw['Company Name'] = jobs_df_raw['Company'].apply(isolate_company_name)


In [4]:
jobs_df_raw['Apply Date'] = pd.to_datetime(jobs_df_raw['Apply Date'])
jobs_df_raw['Reply Date'] = pd.to_datetime(jobs_df_raw['Reply Date'])

In [5]:
# Define a list of column names to keep in the output DataFrame
cols_to_keep = ['Company Name', 'Job', 'Apply Date', 'Reply Date', 'Result',
                'Base Pay', 'Max Pay', 'Link', 'Last Stage', 'Final Decision', 'created']

# Create a copy of the original DataFrame that only includes the desired columns
jobs_df = jobs_df_raw[cols_to_keep].copy()

# For Sankey

In [6]:
filter_job_gone = jobs_df['Result'] == 'Job Gone'
filter_applied = ~pd.isnull(jobs_df['Apply Date'])
filter_replied = ~pd.isnull(jobs_df['Reply Date'])
filter_approved = jobs_df['Result'] == 'Yes'
filter_rejected = jobs_df['Result'] == 'No'
filter_first_screen = jobs_df['Last Stage'] == 'First Screen'
filter_tech_screen = jobs_df['Last Stage'] == 'Tech Screen'
filter_self_removal = jobs_df['Final Decision'] == "Self-removal"
filter_not_selected = jobs_df['Final Decision'] == "Not selected"
filter_ghosted = jobs_df['Final Decision'] == "Ghosted"
filter_in_progress = pd.isnull(jobs_df[filter_approved]['Final Decision'])

In [7]:
job_info = {'total_job_apps': sum(filter_applied | filter_job_gone),
                   'applied': sum(filter_applied),
     'job_gone_before_apply': sum(~filter_applied & filter_job_gone),
                  'response': sum(filter_replied & ~filter_job_gone),
               'no_response': sum(filter_applied & ~filter_replied & ~filter_job_gone),
      'job_gone_after_apply': sum(filter_applied & filter_job_gone),
                  'approved': sum(filter_approved),
                  'rejected': sum(filter_rejected),
              'first screen': sum(filter_first_screen),
               'tech screen': sum(filter_tech_screen)
           }

In [8]:
def generate_outcomes_dict(stage):
    filter_stage = jobs_df['Last Stage'] == stage
    outcome_dict = {'Self Widthdrew': sum(filter_stage & filter_self_removal),
                           'Removed': sum(filter_stage & filter_not_selected),
                           'Ghosted': sum(filter_stage & filter_ghosted),
                       'In Progress': sum(filter_stage & filter_in_progress)}
    return outcome_dict

In [9]:
all_stages = list(jobs_df['Last Stage'].value_counts().keys())

list_of_dicts = {}
for x in all_stages:
    new_dict = generate_outcomes_dict(x)
    list_of_dicts[x] = new_dict

In [10]:
def generate_sankey(stat_df):
    text = ''':Job Applications #adadad
:Applied #7de6f0
:First Screen #75e083

Job Applications [{app}] Applied
Job Applications [{jgba}] Job Gone Before Apply

Applied [{res}] Response
Applied [{nr}] No Response
Applied [{jgaa}] Job Gone After Apply

Response [{ap}] Approved
Response [{rej}] Rejected

Approved [{fs}] First Screen
Approved [{ts}] Tech Screen

'''.format(tja = stat_df['total_job_apps'],
                                    app = stat_df['applied'],
                                    jgba = stat_df['job_gone_before_apply'],
                                    res = stat_df['response'],
                                    nr = stat_df['no_response'],
                                    jgaa = stat_df['job_gone_after_apply'],
                                    ap = stat_df['approved'],
                                    rej = stat_df['rejected'],
                                    fs = stat_df['first screen'],
                                    ts = stat_df['tech screen'])
    
    for x in list_of_dicts:
        for y in list_of_dicts[x]:
            text += f'{x} [{list_of_dicts[x][y]}] {y} \n'
        text+='\n'

    print(text)
    print("https://sankeymatic.com/build/")
generate_sankey(job_info)

:Job Applications #adadad
:Applied #7de6f0
:First Screen #75e083

Job Applications [414] Applied
Job Applications [15] Job Gone Before Apply

Applied [219] Response
Applied [48] No Response
Applied [147] Job Gone After Apply

Response [20] Approved
Response [203] Rejected

Approved [12] First Screen
Approved [6] Tech Screen

First Screen [4] Self Widthdrew 
First Screen [5] Removed 
First Screen [2] Ghosted 
First Screen [1] In Progress 

Tech Screen [2] Self Widthdrew 
Tech Screen [2] Removed 
Tech Screen [1] Ghosted 
Tech Screen [1] In Progress 

On-Site [0] Self Widthdrew 
On-Site [1] Removed 
On-Site [0] Ghosted 
On-Site [0] In Progress 


https://sankeymatic.com/build/
